In [1]:
# Challenge 4: Deep Learning

import tensorflow as tf

# Clears the default graph stack and resets the global default graph.
tf.reset_default_graph() 

In [2]:
from pathlib import Path

from PIL import Image
from PIL import ImageOps
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils  import to_categorical
import tensorflow as tf
data_path = './gear_images_128'
category = 0 # Category label
labels = [] # Label strings
x = [] # Image data
y = [] # Labels for each image

def prepare_dataset(file):    
    try:
        im = Image.open(file)
        arr = np.array(im)
        x.append(arr)
        y.append(to_categorical(category, num_classes=12)) # if category is 1, then it returns [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
    except:
        print("cannot add to dataset:", file)

# Loop all data and create dataset
for dir in Path(data_path).iterdir():
    labels.append(str(dir).split('/')[-1])
    for file in dir.iterdir():
        prepare_dataset(file)
    print(dir, ':', category)
    category += 1

x = np.asarray(x,dtype=float) # Convert image list into array of float.
x = x/255 # Make value between 0 to 1
y = np.asarray(y,dtype=float)

# Split data into 70% train and 30% test. 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

Using TensorFlow backend.


gear_images_128\axes : 0
gear_images_128\boots : 1
gear_images_128\carabiners : 2
gear_images_128\crampons : 3
gear_images_128\gloves : 4
gear_images_128\hardshell_jackets : 5
gear_images_128\harnesses : 6
gear_images_128\helmets : 7
gear_images_128\insulated_jackets : 8
gear_images_128\pulleys : 9
gear_images_128\rope : 10
gear_images_128\tents : 11


In [3]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

IMG_SIZE = 128
LR = 1e-3

# Input data shape is 4-D Tensor. [batch, height, width. channel]
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.5)
convnet = fully_connected(convnet, 12, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
MODEL_NAME  ="AdventureWorksModel-RegressionFirst"
model.fit({'input': x_train}, {'targets': y_train}, n_epoch=10, validation_set=({'input': x_test}, {'targets': y_test}), snapshot_step=10,)

Training Step: 239  | total loss: 0.24054 | time: 83.697s
| Adam | epoch: 010 | loss: 0.24054 -- iter: 1472/1485
Training Step: 240  | total loss: 0.22424 | time: 90.773s
| Adam | epoch: 010 | loss: 0.22424 | val_loss: 0.27118 -- iter: 1485/1485
--


In [18]:
model.save("MODEL_NAME")

INFO:tensorflow:C:\Users\p000524252\jupyter\Challenge4\MODEL_NAME is not in all_model_checkpoint_paths. Manually adding it.


In [19]:
model.load("AdventureWorksModel.tflearn")

INFO:tensorflow:Restoring parameters from C:\Users\p000524252\jupyter\Challenge4\AdventureWorksModel.tflearn


In [20]:
model.evaluate(x_test, y_test)

[0.9215070644577781]

In [29]:
y_predict = model.predict(x_test)

In [48]:
y_test.shape
print(y_test)
y_test_np = np.argmax(y_test,1)
print(y_test_np)
print(y_test_np.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[ 5  1  4  3  4  1  6  2  5  4  4  5 10  8  6  0 11  5  3  9  8  8  2  2
  4  3  6  3 11  3  8 10  2 10  5  2  0  5 11  6  2  3  6  6  2  1  5  5
  3  5  5  6  5  5  9  7  3  7  5  8  8  1  6  8  5  4  0  0  7  2  5  8
 11  2  5 10  6  0  1 10  6  8  2  2  7  6  5  1  5  3  5  3 11 10  2  7
  8  5  5  3  7 11  4  4  4  5  2  5  5  2 11 10  5  8  8  2  2  1  5  6
  5  2  8  0 10  6 10  2  7 11  5  6  6  4  6  3  3  4  5  0  2  5  0  3
  8  5 10  6  8  3  4 10  5  3  8  8  1 10  4  8  8  5  2 11 11  5  6  1
  0  3 10  5  4  6  5  5  5  8  2  2  3  5  8  8  8  3  5 10  2  5  9  8
  0 11  8  7 10  7 11  5 11  2  5 11  2  1  8 11 11  2  1  1  5  9 10  6
  5  2  8  5  2  8  0  5  6  5  9  2 10  9 11 11  9  6  2  2  3  5  7  0
  5  7  2 10  5 11  2  5  2  8  8  1  5 10  5 10 10 11  9  0  8  2  9  8
 10  6  3  7  6  2  6  5  6  3 10 11  6  

In [47]:
y_predict.shape
print(y_predict)
y_pre_np = np.argmax(y_predict,1)
print(y_pre_np)
print(y_pre_np.shape)

[[1.5151639e-07 1.3538017e-05 3.9297065e-05 ... 2.0666688e-05
  1.9951785e-06 1.8902468e-07]
 [5.1288862e-10 9.9990094e-01 4.7375690e-09 ... 2.0696189e-06
  1.4698500e-06 1.4466321e-11]
 [6.8878758e-06 7.5989621e-05 6.5164233e-04 ... 4.4438293e-05
  3.6412304e-05 7.5432780e-07]
 ...
 [4.3061766e-07 1.7510138e-03 2.6194425e-04 ... 1.2893711e-04
  1.6912158e-07 2.5487620e-07]
 [1.4325716e-08 1.3526309e-06 5.3250536e-05 ... 4.0408759e-06
  2.5896325e-08 5.8088881e-11]
 [4.3020293e-07 2.4297477e-05 1.9458799e-04 ... 2.5374376e-05
  7.4143968e-06 6.4430640e-07]]
[ 8  1  4  3  4  1  6  2  5  4  4  5 10  4  6  3 11  5  3  4  8  8  2  2
  4  3  6  3 11  3  8 10  2 10  5  2  0  5 11  6  2  3  6  6  2  1  5  5
  3  5  5  6  5  5 10  7  3  7  5  8  8  1  6  8  8  4  0  0  7  2  5  8
 11  2  5 10  6  0  1 10  6  8  0  2  7  6  4  1  5  3  5  3 11 10  2  7
  8  5  5  3  7 11  4  4  4  5  2  5  5  2 11 10  5  8  8  2  2  1  5  6
  7  2  8  0 10  6 10  2  7  0  5  6  6  4  6  3  3  4  5  0  2  5  0  

In [49]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test_np,y_pre_np)

array([[ 28,   0,   1,   2,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,  28,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1],
       [  1,   1,  75,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,  45,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,  63,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   2, 114,   0,   1,  15,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,  52,   0,   0,   0,   0,   0],
       [  0,   0,   1,   0,   3,   0,   0,  26,   0,   0,   0,   0],
       [  0,   0,   0,   0,   4,   9,   0,   0,  70,   0,   0,   0],
       [  1,   0,   2,   0,   1,   0,   0,   0,   0,   9,   1,   0],
       [  0,   0,   0,   0,   1,   0,   0,   0,   0,   0,  47,   0],
       [  2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,  30]],
      dtype=int64)

In [50]:
from sklearn.metrics import classification_report

print(classification_report(y_test_np, y_pre_np))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89        31
           1       0.97      0.97      0.97        29
           2       0.95      0.97      0.96        77
           3       0.96      1.00      0.98        45
           4       0.85      1.00      0.92        63
           5       0.93      0.86      0.89       132
           6       1.00      1.00      1.00        52
           7       0.96      0.87      0.91        30
           8       0.82      0.84      0.83        83
           9       1.00      0.64      0.78        14
          10       0.96      0.98      0.97        48
          11       0.97      0.91      0.94        33

   micro avg       0.92      0.92      0.92       637
   macro avg       0.94      0.91      0.92       637
weighted avg       0.92      0.92      0.92       637



In [58]:
predict_classes = model.predict_classes(x_test, batch_size=1)
prediction = []
prediction.append(y_predict)
labels.append(y_test)
confusionMat=tf.confusion_matrix(y_test,y_predict,2)

AttributeError: 'DNN' object has no attribute 'predict_classes'